In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):

    def __init__(self, in_channels, out_channels, shape):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(shape),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):

    def __init__(self, in_channels, out_channels, shape):
        super().__init__()

        self.up = nn.Upsample(scale_factor=shape)
        self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1):
        x1 = self.up(x1)
        return self.conv(x1)


class OutLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutLayer, self).__init__()
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x):
        return self.linear(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class MiddlePad(nn.Module):
    def __init__(self, padding):
        super(MiddlePad, self).__init__()
        self.pad = nn.ZeroPad2d(padding)
    
    def forward(self, x):
        return self.pad(x)


class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.inc = DoubleConv(n_channels, 8)
        self.down1 = Down(8, 16, (4, 2))
        self.down2 = Down(16, 32, (3, 2))
        self.down3 = Down(32, 64, (3, 2))
        self.down4 = Down(64, 128, (3, 2))
        self.pad = MiddlePad((0, 0, 1, 1))
        self.up1 = Up(128, 64, 2)
        self.up2 = Up(64, 32, 2)
        self.up3 = Up(32, 16, 2)
        self.up4 = Up(16, 8, 2)
        self.convout = OutConv(8, 1)

    def forward(self, x):
        x = self.inc(x)
        x = self.down1(x)
        x = self.down2(x)
        x = self.down3(x)
        x = self.down4(x)
        x = self.pad(x)
        x = self.up1(x)
        x = self.up2(x)
        x = self.up3(x)
        x = self.up4(x)
        x = self.convout(x)
        return x